In [1]:
import pandas as pd

In [2]:
# Histórico dos preços de fechamento do HGLG11
hglg_precos = pd.read_csv('data/hglg_precos.csv', parse_dates=['data'])
hglg_precos

,data,preco
0,2011-03-02,104.40
1,2011-03-21,104.40
2,2011-03-28,104.40
3,2011-03-31,104.40
4,2011-04-05,104.40
...,...,...
2767,2022-12-05,162.84
2768,2022-12-06,162.95
2769,2022-12-07,162.18
2770,2022-12-08,162.20


In [6]:
# Histórico de dividendos do HGLG11 em que a data base foi ajustada
# para ser necessariamente em dia com pregão
hglg_divs = (
    pd.read_csv('data/hglg_divs.csv', parse_dates=['data'])
    # Remover as colunas que não serão utilizadas
    .drop(columns=['tipo', 'data_pagamento'])
)
hglg_divs

,data,dividendo
0,2011-03-31,0.72
1,2011-04-29,0.75
2,2011-05-31,0.79
3,2011-06-30,0.80
4,2011-07-29,0.81
...,...,...
136,2022-07-29,1.10
137,2022-08-31,1.10
138,2022-09-30,1.10
139,2022-10-31,1.10


In [7]:
def ajustar_precos(df_precos: pd.DataFrame, df_divs: pd.DataFrame) -> pd.DataFrame:
    """Ajusta o preço do ativo de acordo com o valor do dividendo pago.
    
    Parâmetros:
        df_precos: dataFrame com as colunas "data" e "preco" do ativo
        df_divs: dataFrame com as colunas "data" e "dividendo", sendo "data"
                 a data base do dividendo pago (data anterior à data ex-dividendo)
    
    Retorno: dataFrame com as colunas "data" e "preco" ajustado para trás de acordo com  
             o dividendo pago (preço mais recente inalterado).
             Demais colunas do dataframe são mantidas inalteradas.
    """
    # Ajustar as datas de dividendos para que a data tenha tido necessariamente negociação
    df_divs['data'] = df_divs['data'].apply(
        lambda x: df_precos.query('data <= @x')['data'].max()
    )
    # Incorporar os dados de dividendos em df_precos
    df_precos = df_precos.merge(df_divs, how='left')
    # Calcular o fator de ajuste
    df_precos["ajuste"] = 1 - df_precos["dividendo"] / df_precos["preco"]
    # Preencher os valores faltantes com 1 para o cálculo do acumulado
    df_precos["ajuste"] = df_precos["ajuste"].fillna(1)
    # Ordenar as datas da mais nova p/ a mais antiga -> preço inalterado na data mais recente
    df_precos.sort_values("data", ascending=False, ignore_index=True, inplace=True)
    # Calcular o acumulado do fator de ajuste
    df_precos["ajuste_acum"] = df_precos["ajuste"].cumprod()
    # Ajustar o preço do ativo
    df_precos["preco"] = df_precos["preco"] * df_precos["ajuste_acum"]
    # Remover colunas criadas para o cálculo do ajuste
    df_precos.drop(columns=["ajuste", "ajuste_acum"], inplace=True)
    # Voltar a ordenar as datas da mais antiga p/ a mais nova
    df_precos.sort_values("data", ascending=True, ignore_index=True, inplace=True)
    return df_precos

In [8]:
ajustar_precos(hglg_precos, hglg_divs)
hglg_precos_ajustados = ajustar_precos(hglg_precos, hglg_divs)
hglg_precos_ajustados

,data,preco,dividendo
0,2011-03-02,37.450552,NaN
1,2011-03-21,37.450552,NaN
2,2011-03-28,37.450552,NaN
3,2011-03-31,37.450552,0.72
4,2011-04-05,37.710625,NaN
...,...,...,...
2767,2022-12-05,162.840000,NaN
2768,2022-12-06,162.950000,NaN
2769,2022-12-07,162.180000,NaN
2770,2022-12-08,162.200000,NaN


In [9]:
hglg_precos_ajustados.query('data >= "2022-03-30"')

,data,preco,dividendo
2596,2022-03-30,152.426250,NaN
2597,2022-03-31,151.961536,1.1
2598,2022-04-01,152.625900,NaN
2599,2022-04-04,153.177977,NaN
2600,2022-04-05,153.458694,NaN
...,...,...,...
2767,2022-12-05,162.840000,NaN
2768,2022-12-06,162.950000,NaN
2769,2022-12-07,162.180000,NaN
2770,2022-12-08,162.200000,NaN


In [11]:
# Comparar os preços com e sem ajuste do HGLG11 com o Plotly
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=hglg_precos["data"],
        y=hglg_precos["preco"],
        mode="lines",
        name="Preço sem ajuste",
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg_precos_ajustados["data"],
        y=hglg_precos_ajustados["preco"],
        mode="lines",
        name="Preço com ajuste",
    )
)
fig.update_layout(
    title="Preços do HGLG11 com e sem ajuste",
    xaxis_title="Data",
    yaxis_title="Preço",
    legend_title="",
)
fig.show()

In [12]:
# Carregar os dados de preços e dividendos do PETR4
petr_precos = (
    pd.read_csv('data/petr_precos.csv', parse_dates=['data'])
    # Concidir com o período de negociação do HGLG11
    .query('data >= "2011-03-02"')
    # Renomear com o preço desejado para "preco"
    .rename(columns={'preult': 'preco'})
    # Selecionar apenas as colunas de interesse
    [['data', 'preco']]
)
petr_divs = pd.read_csv('data/petr_divs.csv', parse_dates=['data'])
petr_divs[-5:]

,data,dividendo
45,2021-12-02,3.250487
46,2022-04-14,2.970249
47,2022-05-24,3.715490
48,2022-08-12,6.732003
49,2022-11-22,3.348900


In [13]:
# Ajustart os preços de fechamento da PETR4
petr_precos_ajustados = ajustar_precos(petr_precos, petr_divs)
petr_precos_ajustados

,data,preco,dividendo
0,2011-03-02,10.036748,NaN
1,2011-03-03,10.057585,NaN
2,2011-03-04,10.099260,NaN
3,2011-03-09,10.099260,NaN
4,2011-03-10,9.765860,NaN
...,...,...,...
2920,2022-12-07,25.350000,NaN
2921,2022-12-08,24.780001,NaN
2922,2022-12-09,24.709999,NaN
2923,2022-12-12,23.910000,NaN


In [14]:
# Comparar os preços ajustados e normalizados (base 100) para o início da série dos dois ativos com o Plotly
import plotly.express as px

# Normalizar os preços para base 100
hglg_precos_ajustados['preco'] = hglg_precos_ajustados['preco'] / hglg_precos_ajustados['preco'].iloc[0] * 100
petr_precos_ajustados['preco'] = petr_precos_ajustados['preco'] / petr_precos_ajustados['preco'].iloc[0] * 100

# Juntar os preços ajustados e normalizados em um único DataFrame
df = pd.concat([hglg_precos_ajustados, petr_precos_ajustados], ignore_index=True)
df['ativo'] = ['HGLG11'] * len(hglg_precos_ajustados) + ['PETR4'] * len(petr_precos_ajustados)

# Plotar os preços ajustados e normalizados
fig = px.line(df, x='data', y='preco', color='ativo', title='Preços ajustados e normalizados')
fig.show()
